Before you turn this project in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Paul Tluczek"
COLLABORATORS = ""

# Load Data

The following code loads in the MNIST dataset and displays a few images and flattens the images and sets some autograder variables.

In [2]:
from IPython.display import display
import numpy as np
from keras.preprocessing.image import array_to_img, img_to_array
from keras.datasets import mnist
from keras.utils.np_utils import to_categorical
%load_ext autoreload
%autoreload

[X, y], _ = mnist.load_data()
for x in X[:5]:
    x = np.expand_dims(x, axis=-1)
    img = array_to_img(x)
    display(img)

X = X.reshape([60_000, 28*28]) / 255.
Y = to_categorical(y)
X, Y = X[:50], Y[:50]

M, N = X.shape
C = np.unique(y).shape[0]
H = 16

def passed(): print('✅')

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.


# Task

- Implement `Dense`, `Sigmoid`, and `SoftmaxCE` layers as classes and stick them in a file `layers.py`

In [3]:
%%writefile layers.py

# YOUR CODE HERE
import numpy as np
class Dense:
    def __init__(self, N, H):
        self.W = np.random.randn(N, H)
        self.b = np.random.randn(H)
    def forward(self, X):
        Z = np.dot(X,self.W) + self.b
        self.X = X
        self.cache = locals()             
        return Z
    def backward(self, dZ):
        X = self.cache['X']        
        db = dZ.sum(axis=0)
        dX, dW = np.dot(dZ,np.transpose(self.W)), np.dot(np.transpose(X),dZ)
        return dX, dW, db

class Sigmoid:
    def forward(self, Z):
        H = 1 / (1 + np.exp(-Z))
        self.cache = locals()        
        return H
    def backward(self, dH):
        H = self.cache['H']        
        dZ = H * (1-H) * dH
        return dZ
    
class SoftmaxCE:
    def forward(self, S, Y):
        P = np.exp(S) / (np.exp(S).sum(axis=1, keepdims=True))
        y = Y.argmax(axis=1)
        M = len(P)
        L = P[np.arange(M), y]
        L = -np.log(L)
        L = np.expand_dims(L, axis=-1)
        self.cache = locals()
        return L
    def backward(self, dL):
        P, y, M = self.cache['P'], self.cache['y'], self.cache['M']
        dLdS = P
        dLdS[np.arange(M), y] -= 1
        dS = dLdS * dL
        return dS  

Writing layers.py


# Task

- Define a one-hidden layer perceptron class called `LayeredMLP` which uses Dense, Sigmoid, and SoftmaxCE layers as in the computational graph

![](images/mlp_predict.svg)

where

- $\mathbf{X} \in \mathbb{R}^{M \times N}$
- $\mathbf{W}^{(1)} \in \mathbb{R}^{N \times H}$ and $\mathbf{b}^{(1)} \in \mathbb{R}^{H}$
- $\mathbf{Z} \in \mathbb{R}^{M \times H}$
- $\mathbf{H} \in \mathbb{R}^{M \times H}$
- $\mathbf{W}^{(2)} \in \mathbb{R}^{H \times C}$ and $\mathbf{b}^{(2)} \in \mathbb{R}^{C}$
- $\mathbf{S} \in \mathbb{R}^{M \times C}$
- $\mathbf{Y} \in \mathbb{R}^{M \times C}$
- $\mathbf{L} \in \mathbb{R}^{M}$ and $\overline{\ell} \in \mathbb{R}$

In [4]:
%%writefile classifiers.py

# YOUR CODE HERE
import numpy as np
import layers
       
class LayeredMLP():
    def __init__(self, nb_feature, nb_hidden, nb_class):        
        self.dense1 = layers.Dense(N=nb_feature, H=nb_hidden)
        self.sigmoid = layers.Sigmoid()
        self.dense2 = layers.Dense(N=nb_hidden, H=nb_class)
        self.softmaxce = layers.SoftmaxCE()
        
    def _f(self, X):
        Z = self.dense1.forward(X) 
        H = self.sigmoid.forward(Z)
        S = self.dense2.forward(H)        
        self.cache = locals()
        
    def predict(self, X):
        self._f(X)
        S = self.cache['S']
        P = np.exp(S) / np.exp(S).sum(axis=1, keepdims=True)
        return P

    def evaluate(self, X, Y):
        return np.mean((self.predict(X)).argmax(axis=1) == Y.argmax(axis=1))  
    
    def forward(self, X, Y):
        self._f(X)
        S = self.cache['S']
        L = self.softmaxce.forward(S, Y)        
        return np.mean(L)    

Overwriting classifiers.py


# Constructor Tests

In [5]:
%autoreload
from classifiers import LayeredMLP
import layers

mlp = LayeredMLP(nb_feature=N, nb_hidden=H, nb_class=C)

assert type(mlp.dense1) == layers.Dense
assert hasattr(mlp.dense1, 'W')
assert hasattr(mlp.dense1, 'b')
assert type(mlp.sigmoid) == layers.Sigmoid
assert type(mlp.dense2) == layers.Dense
assert hasattr(mlp.dense2, 'W')
assert hasattr(mlp.dense2, 'b')
assert type(mlp.softmaxce) == layers.SoftmaxCE

passed()

✅


# Prediction Tests

In [6]:
from classifiers import LayeredMLP

mlp = LayeredMLP(nb_feature=N, nb_hidden=H, nb_class=C)

S = mlp.predict(X)
nb_train_ = len(X)
assert S.shape == (M, C)

passed()

✅


# Evaluation Tests

In [7]:
from classifiers import LayeredMLP

mlp = LayeredMLP(nb_feature=N, nb_hidden=H, nb_class=C)

acc = mlp.evaluate(X, Y)
assert type(acc) == np.float64
assert 0 <= acc <= 1

passed()

✅


# Task

- Implement a `LayeredMLPWithGDOptimizer` class which performs optimization via gradient descent and extends your `LayeredMLP` class

# Requirement

- You must use backpropagation to compute gradients. To demonstrate this I am requiring your `_get_gradients()` function needs to return the gradient of every intermediate value in the computational graph as in

![](images/mlp_full.svg)

including `dX` (not pictured). You don't have to return `dloss`. Check the tests below to clear up any confusion.

In [8]:
%%writefile -a classifiers.py

# YOUR CODE HERE
class LayeredMLPWithGDOptimizer(LayeredMLP):
    def __init__(self, nb_feature, nb_hidden, nb_class, alphalr=0.01):        
        LayeredMLP.__init__(self, nb_feature, nb_hidden, nb_class) 
        self.alphalr = alphalr
 
    def fit(self, X, Y, nb_epoch=1):
        for _ in range(nb_epoch):
            # Store current parameters
            curr_W1, curr_b1, curr_W2, curr_b2 = self.dense1.W, self.dense1.b, self.dense2.W, self.dense2.b           
            # Store current parameter loss
            curr_loss = self.forward(X, Y) 
 
            dL = np.ones([len(X), 1], dtype='float64')
            dS = self.softmaxce.backward(dL)    
            dH, dW2, db2 = self.dense2.backward(dS)
            dZ = self.sigmoid.backward(dH)
            dX, dW1, db1 = self.dense1.backward(dZ)
            
            # New parameters after GD
            self.dense1.W = self.dense1.W - (self.alphalr * dW1)
            self.dense1.b = self.dense1.b - (self.alphalr * db1)
            self.dense2.W = self.dense2.W - (self.alphalr * dW2)
            self.dense2.b = self.dense2.b - (self.alphalr * db2)
            
            new_loss = self.forward(X, Y)

            # If new parameters give higher loss, revert to old parameters 
            if curr_loss < new_loss:                               
                self.dense1.W, self.dense1.b, self.dense2.W, self.dense2.b = curr_W1, curr_b1, curr_W2, curr_b2  
    
    def _get_gradients(self, X, Y):
        curr_loss = self.forward(X, Y) 
        dL = np.ones([len(X), 1], dtype='float64')
        dS = self.softmaxce.backward(dL)    
        dH, dW2, db2 = self.dense2.backward(dS)
        dZ = self.sigmoid.backward(dH)
        dX, dW1, db1 = self.dense1.backward(dZ)
        return dX, dW1, db1, dZ, dH, dW2, db2, dS                



Appending to classifiers.py


# Gradient Checking Tests

In [9]:
%autoreload
from classifiers import LayeredMLP, LayeredMLPWithGDOptimizer

mlp = LayeredMLPWithGDOptimizer(nb_feature=N, nb_hidden=H, nb_class=C)
assert issubclass(LayeredMLPWithGDOptimizer, LayeredMLP)

gradients = mlp._get_gradients(X, Y)
for gradient in gradients:
    assert type(gradient) == np.ndarray

In [10]:
import logging

# Gradient Checking Tests

In [11]:
from classifiers import LayeredMLPWithGDOptimizer
from checking import estimate_gradients

mlp = LayeredMLPWithGDOptimizer(nb_feature=N, nb_hidden=H, nb_class=C)

estimated_gradients = estimate_gradients(mlp, X, Y)
dX, dW1, db1, dZ, dH, dW2, db2, dS = mlp._get_gradients(X, Y)
analytical_gradients, params = [dW1, db1, dW2, db2], ['dW1', 'db1', 'dW2', 'db2']
grad_pairs = zip(estimated_gradients, analytical_gradients, params)
for i, (estimated_gradient, analytic_gradient, param) in enumerate(grad_pairs):
    try:
        assert np.allclose(estimated_gradient, analytic_gradient)
    except:
        norm = np.square(estimated_gradient - analytic_gradient).mean()
        logging.warning(f'{param} check failed with a difference of {norm}!')

# Gradient Descent Optimizer Tests

In [12]:
from classifiers import LayeredMLPWithGDOptimizer

mlp = LayeredMLPWithGDOptimizer(nb_feature=N, nb_hidden=H, nb_class=C)

X_sample, Y_sample = X[:50], Y[:50]
acc = mlp.evaluate(X_sample, Y_sample)
loss = mlp.forward(X_sample, Y_sample)
for _ in range(10):
    mlp.fit(X_sample, Y_sample, nb_epoch=10)
    assert mlp.forward(X_sample, Y_sample) < loss
    loss = mlp.forward(X_sample, Y_sample)
    
assert mlp.evaluate(X_sample, Y_sample) > acc

passed()

✅


# Ignore Cell Below

In [13]:
# YOUR CODE HERE
